In [1]:
import vtk
import numpy as np
import pandas as pd
import time
import re
import threading
from sksurgerynditracker.nditracker import NDITracker

In [2]:
# Initialize the NDI tracker
SETTINGS = {
    "tracker type": "vega",
    "ip address": "169.254.7.250",
    "port": 8765,
    "romfiles": ["/Users/yizheng/Desktop/Dartmouth/lab work/ARPAH/Polaris Vega XT/tool_defs/medtronic_chicken_foot_960_556.rom"]
}
tracker = NDITracker(SETTINGS)
tracker.start_tracking()

PING 169.254.7.250 (169.254.7.250): 56 data bytes
64 bytes from 169.254.7.250: icmp_seq=0 ttl=64 time=0.756 ms

--- 169.254.7.250 ping statistics ---
1 packets transmitted, 1 packets received, 0.0% packet loss
round-trip min/avg/max/stddev = 0.756/0.756/0.756/0.000 ms


In [3]:
# Initialize vtk
reader = vtk.vtkSTLReader() # read stl file
reader.SetFileName("medtronic_chicken_foot.STL") # set the file name
reader.Update() # update the reader

mapper = vtk.vtkPolyDataMapper() # create a mapper, a mapper is used to map the data to the graphics primitives
mapper.SetInputData(reader.GetOutput()) # set the input to the reader

actor = vtk.vtkActor() # create an actor, an actor is used to display the data
actor.SetMapper(mapper) # set the mapper to the actor

renderer = vtk.vtkRenderer() # create a renderer, a renderer is used to display the data
render_window = vtk.vtkRenderWindow() # create a render window, a render window is used to display the data
render_window.AddRenderer(renderer) # add the renderer to the render window
render_window.SetSize(2000, 1600) # set the size of the render window

render_window_interactor = vtk.vtkRenderWindowInteractor() # create a render window interactor, a render window interactor is used to interact with the data
render_window_interactor.SetRenderWindow(render_window) # set the render window to the render window interactor

renderer.AddActor(actor) # add the actor to the renderer
renderer.SetBackground(0.1, 0.2, 0.4) # set the background of the renderer

# 创建坐标轴
axes = vtk.vtkAxesActor()
axes.SetTotalLength(200, 200, 200)  # 设置坐标轴的长度
axes.SetAxisLabels(True)

# 设置坐标轴标签的位置
axes.GetXAxisCaptionActor2D().GetTextActor().GetPositionCoordinate().SetCoordinateSystemToNormalizedDisplay()
axes.GetYAxisCaptionActor2D().GetTextActor().GetPositionCoordinate().SetCoordinateSystemToNormalizedDisplay()
axes.GetZAxisCaptionActor2D().GetTextActor().GetPositionCoordinate().SetCoordinateSystemToNormalizedDisplay()

# 设置坐标轴标签
axes.GetXAxisCaptionActor2D().GetTextActor().GetProperty().SetColor(1, 0, 0)  # X轴为红色
axes.GetYAxisCaptionActor2D().GetTextActor().GetProperty().SetColor(0, 1, 0)  # Y轴为绿色
axes.GetZAxisCaptionActor2D().GetTextActor().GetProperty().SetColor(0, 0, 1)  # Z轴为蓝色

# 将坐标轴添加到 renderer
renderer.AddActor(axes)

# 设定摄像机视角，固定在 (0, 0, -3000) 位置
renderer.GetActiveCamera().SetPosition(-1000, 1000, -5000)  # 摄像机位置
renderer.GetActiveCamera().SetFocalPoint(0, 0, 0)  # 摄像机焦点
renderer.GetActiveCamera().SetViewUp(-1, 0, 0)  # 设置“上”方向
renderer.GetActiveCamera().SetClippingRange(1, 3000)  # 设置裁剪范围

In [4]:
def get_tracking_data(tracker):
    """Fetch latest tracking data"""
    port_handles, timestamps, framenumbers, transformation, quality = tracker.get_frame()
    transformation = np.array(transformation[0])
    return timestamps, transformation, quality

In [5]:
# Polaris Data Streaming Thread
data_ready = threading.Event()

def polaris_thread():
    global polaris_matrix
    while True:
        _, new_matrix, _ = get_tracking_data(tracker)
        polaris_matrix = new_matrix
        data_ready.set()
        time.sleep(1/400)

In [6]:
# VTK Update Thread
def update_pose(obj, event):
    global polaris_matrix

    if data_ready.is_set():
        transform = vtk.vtkTransform()
        transform.SetMatrix(polaris_matrix.flatten()) 
        actor.SetUserTransform(transform)
        render_window.Render()
        data_ready.clear()

: 

In [ ]:
_, transformation, _ = get_tracking_data(tracker)

threading.Thread(target=polaris_thread, daemon=True).start()

render_window_interactor.AddObserver("TimerEvent", update_pose)
render_window_interactor.CreateRepeatingTimer(1)  # VTK 线程不断监听

render_window.Render()
render_window_interactor.Initialize()
render_window_interactor.Start()

2025-02-19 17:26:10.722 python[6644:132633] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-19 17:26:10.723 python[6644:132633] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [ ]:
transformation